# Executar GWAS usando GMMAT
**Autores:** Rafaella Ormond e Jose Jaime Martinez-Magana  
**Data:** 25 de julho de 2025  

Este script foi desenvolvido para rodar GMMAT em coortes do LAGC.  

**ENTRADA:**
- Dados de genótipo no formato PLINK pgen/pvar/sample (.pgen, .pvar, .sample)
- Arquivos de fenótipos e covariáveis correspondentes à coorte
- Covariáveis para ajuste na GWAS
- Matriz GRM (estimada no script PCAir: [LINK AQUI](https://github.com/ormondr/Smoking_GWAS_LAGC/blob/main/English/01PC/00PCAir.ipynb))

**SAÍDA:** 
- Resultados de associação GWAS do GMMAT (arquivos GMMAT) com estatísticas resumidas

### Requisitos

### Baixar e instalar R

Você pode baixar e instalar R a partir do Comprehensive R Archive Network (CRAN) para o seu sistema operacional:

- [R para Windows](https://cran.r-project.org/bin/windows/base/)
- [R para macOS](https://cran.r-project.org/bin/macosx/)
- [R para Linux](https://cran.r-project.org/bin/linux/)
  
Após instalar R, você pode instalar os pacotes necessários conforme mostrado abaixo.

### Baixar GMMAT
GMMAT é um pacote R para análise de associação genética. Para instalar GMMAT, é necessário ter R (versão 3.5 ou superior) e o pacote devtools instalado.  
[Acessar o github do GMMAT aqui](https://github.com/hanchenphd/GMMAT)<br>
ou<br>
Execute os seguintes comandos no R:

`if (!requireNamespace("devtools", quietly = TRUE))`  

`install.packages("devtools")`  

`devtools::install_github("jiehuang2020/GMMAT")`

### Instalar outros pacotes no R
`install.packages("optparse")`  

`install.packages("plyr")`

Este método é recomendado principalmente para amostras com menos de 300 indivíduos [(ver plano de análise)](https://docs.google.com/document/d/1RzD5kBlj9rfiomda1G3NfxYDXLdmIUO7VX0cSNj70Kk/edit?usp=sharing).

### Etapas da Análise:
1) Construir modelos nulos  
2) Rodar testes de associação usando o teste de escore do GMMAT


### 1. Construir Modelos Nulos

**Descrição:**  
Este script constrói o **modelo nulo** usando o **GMMAT**.  
O modelo inclui **10 componentes principais (PCs)**, **idade** e a **matriz de relação genética (GRM)** como covariáveis.  
Será necessário ajustar os caminhos dos arquivos de entrada e os nomes das variáveis de acordo com o seu conjunto de dados antes de executar o script.


In [ ]:
########### 
# Este script precisa ser ajustado para os seus arquivos
###########

# carregar GMMAT
library(GMMAT)

# ler arquivo de fenótipos
# o arquivo de fenótipos precisa ter FID, IID e todos os fenótipos desta coorte
# masculino
pheno_male=read.table("/caminho/para_seus_dados/cohort_name_pheno_male.txt", header= T)
# feminino
pheno_female=read.table("/caminho/para_seus_dados/cohort_name_female.txt", header= T)

# ler PCA e GRM
grm=readRDS("/caminho/para_seus_dados/cohort_name.gds_prunned_grm_pca.rds")

# unir PCs para o modelo
pcs=grm$PCair$vectors[,c(1:10)]
pcs=as.data.frame(pcs)
colnames(pcs)=paste0(rep("PC",10),rep(1:10))

# adicionar coluna SampleID
pcs$SampleID=rownames(pcs)

# criar coluna com SampleID (mesmo que IID)
# masculino
pheno_male$SampleID=pheno_male$IID
# feminino
pheno_female$SampleID=pheno_female$IID

# unir dados
library(plyr)
# masculino
pheno_pcs_male <- join_all(list(pheno_male, pcs), by = "SampleID", type = "inner")
# feminino
pheno_pcs_female <- join_all(list(pheno_female, pcs), by = "SampleID", type = "inner")

# subsetting GRM
# masculino
grm_subset_male=grm$grm_sparse[,rownames(grm$grm_sparse) %in% pheno_pcs_male$SampleID]
grm_subset_male=grm_subset_male[rownames(grm$grm_sparse) %in% pheno_pcs_male$SampleID,]
# feminino
grm_subset_female=grm$grm_sparse[,rownames(grm$grm_sparse) %in% pheno_pcs_female$SampleID]
grm_subset_female=grm_subset_female[rownames(grm$grm_sparse) %in% pheno_pcs_female$SampleID,]

# verificar ordem da matriz GRM e do pheno
# extrair índice de ordenação
# masculino
reorde_idx_male=match(rownames(grm_subset_male), pheno_pcs_male$SampleID)
# feminino
reorde_idx_female=match(rownames(grm_subset_female), pheno_pcs_female$SampleID)

# reordenar pheno com base no índice
# masculino
pheno_pcs_male_or=pheno_pcs_male[reorde_idx_male,]
# feminino
pheno_pcs_female_or=pheno_pcs_female[reorde_idx_female,]

### Fenótipos Quantitativos
# Substitua aqui pelos fenótipos quantitativos que você possui nesta coorte (exemplo: "AgeSmkInit")
# obter fenótipos para modelos lineares
# para variáveis lineares (quantitative traits)
# para fenótipos quantitativos usamos distribuição Gaussiana. Certifique-se que "family" na função lmmkin seja: gaussian(link = "identity")
# esta etapa será executada separadamente para cada fenótipo

## Idade de Início do Tabagismo (AgeSmkInit)
# masculino
model_null_AgeSmkInit_10pcs_male=glmmkin(fixed = AgeSmkInit ~ age + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
                                      data=pheno_pcs_male_or,
                                      id="SampleID",
                                      kins=grm_subset_male,
                                      family=gaussian(link = "identity"))

# feminino
model_null_AgeSmkInit_10pcs_female=glmmkin(fixed = AgeSmkInit ~ age + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
                                      data=pheno_pcs_female_or,
                                      id="SampleID",
                                      kins=grm_subset_female,
                                      family=gaussian(link = "identity"))

### Fenótipos Binários
# Substitua aqui pelos fenótipos binários que você possui nesta coorte (exemplo: "SmkCes")
# obter fenótipos para modelos logísticos
# ajuste da codificação para Binary Traits
# esta etapa será executada separadamente para cada fenótipo
# para fenótipos binários usamos distribuição Binomial. Certifique-se que "family" na função lmmkin seja: binomial(link = "logit")

## Cessação do Tabagismo (SmkCes)
# masculino
model_null_SmkCes_10pcs_male=glmmkin(fixed = SmkCes ~ age + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
                   data=pheno_pcs_male_or,
                   id="SampleID",
                   kins=grm_subset_male,
                   family=binomial(link = "logit"))
# feminino
model_null_SmkCes_10pcs_female=glmmkin(fixed = SmkCes ~ age + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
                   data=pheno_pcs_female_or,
                   id="


### 2.1.1 Salvando os Modelos Nulos

Nesta etapa, salvamos os **modelos nulos** que serão utilizados na segunda etapa do GMMAT para os testes de associação.<br>
Os modelos serão salvos como objetos **RDS**, que podem ser carregados posteriormente no R.

In [ ]:
# Salvar os Modelos Nulos

# Salvar modelos nulos lineares para Fenótipos Quantitativos
# Substitua o caminho conforme necessário
# masculino
AgeSmkInit_null_10pcs_male="/path_to_your_data/model_null_AgeSmkInit_10pcs_male.rds"
saveRDS(file=AgeSmkInit_null_10pcs_male,
        model_null_AgeSmkInit_10pcs_male)
# feminino
AgeSmkInit_null_10pcs_female="/path_to_your_data/model_null_AgeSmkInit_10pcs_female.rds"
saveRDS(file=AgeSmkInit_null_10pcs_female,
        model_null_AgeSmkInit_10pcs_female)

# Salvar modelos nulos lineares para Fenótipos Binários
# masculino
SmkCes_null_10pcs_male="/path_to_your_data/model_null_SmkCes_10pcs_male.rds"
saveRDS(file=SmkCes_null_10pcs_male,
        model_null_SmkCes_10pcs_male)
# feminino
SmkCes_null_10pcs_female="/path_to_your_data/model_null_SmkCes_10pcs_female.rds"
saveRDS(file=SmkCes_null_10pcs_female,
        model_null_SmkCes_10pcs_female)

# sair do R
q()


### 2. Score com GMMAT


### 2.1 Construir Script R para rodar GMMAT scoring
Rscript: gmmat_wrapper.Rscript<br>
Este script pode ser baixado no GitHub [LINK AQUI](https://github.com/ormondr/Smoking_GWAS_LAGC/blob/main/English/02GWAS/01GMMAT/gmmat_wrapper.Rscript)

In [ ]:
####################################################################################
# Script para rodar o teste de scoring GMMAT em arquivos PLINK
# Data: 25 jul 2025
# Autores: Rafaella Ormond e Jose Jaime Martinez-Magana
####################################################################################
# Este script irá rodar o GMMAT usando os modelos nulos previamente construídos
####################################################################################
# Carregando bibliotecas
library(optparse)
library(GMMAT)
####################################################################################
# Definindo parâmetros
# Esta função usa a biblioteca optparse para adicionar argumentos ao script
# Adicionando argumentos ao script
option_list = list(
    make_option(c("--nullmodel_path"), type="character", default=NULL,
                help="Caminho completo para o objeto RDS com o modelo nulo construído com GMMAT. Exemplo: /data/nul_model/null_model_hgt.rds ", metavar="character"),
    make_option(c("--genofile"), type="character", default=NULL,
                help="Caminho para os arquivos PLINK. Exemplo: /data/example.bed", metavar="character"),
    make_option(c("--remove_fid"), type="character", default=NULL,
                help="TRUE se o script deve separar o FID e IID no modelo nulo, baseado no underscore _. Exemplo: FID_IID para manter apenas IID", metavar="character"),
    make_option(c("--outfile_path"), type="character", default=NULL,
                help="Nome do arquivo de saída para os resultados de estatísticas de associação em formato tsv. Exemplo: /data/analysis/results_ancestry_specific.tsv", metavar="character")
);
# Configurando parâmetros
opt_parser = OptionParser(option_list=option_list);
opt = parse_args(opt_parser);
####################################################################################
# Lendo dados
# Lendo modelos nulos
model0=readRDS(opt$nullmodel_path)
# Definindo caminho para arquivo genotípico
geno_file=opt$genofile
# Definindo caminho para salvar resultados de estatísticas
outfile=opt$outfile_path

# Separando FID e IID
if(opt$remove_fid){
    print(paste0("Removendo FID do modelo nulo baseado em _"))
    fid_id_s=strsplit(model0$id_include, "_")
    fid_id_splitted=c()
    for(i in 1:length(fid_id_s)){
        id=fid_id_s[[i]][2]
        fid_id_splitted=c(fid_id_splitted, id)
    }
    model0$id_include=fid_id_splitted
} else {
    model0$id_include=model0$id_include
}

# Rodando teste de scoring com GMMAT
glmm.score(model0,
           infile=geno_file,
           outfile=outfile)


### 2.1 Executando os testes de scoring com GMMAT
**Descrição:**<br>
Este script executa os testes de associação usando o script previamente construído.

In [ ]:
### Substitua aqui os caminhos e arquivos
### Substitua pelos fenótipos que você possui na coorte

## Idade de Início do Tabagismo (AgeSmkInit)
## masculino
gmmat_wrapper="gmmat_wrapper.Rscript"
AgeSmkInit_male_models="/caminho_para_seus_dados/model_null_AgeSmkInit_10pcs_male.rds"
genofile="/caminho_para_seus_dados/cohort_name.allchr-merge"
outfile="/caminho_para_seus_dados/coorte/AgeSmkInit_male_score.txt"
# executando o script
Rscript ${gmmat_wrapper} --nullmodel_path=${AgeSmkInit_male_models} \
--genofile=${genofile} \
--remove_fid=FALSE \
--outfile=${outfile}

## feminino
gmmat_wrapper="gmmat_wrapper.Rscript"
AgeSmkInit_female_models="/caminho_para_seus_dados/model_null_AgeSmkInit_10pcs_female.rds"
genofile="/caminho_para_seus_dados/cohort_name.allchr-merge"
outfile="/caminho_para_seus_dados/AgeSmkInit_female_score.txt"
# executando o script
Rscript ${gmmat_wrapper} --nullmodel_path=${AgeSmkInit_female_models} \
--genofile=${genofile} \
--remove_fid=FALSE \
--outfile=${outfile}

## Cessação do Tabagismo (SmkCes)
## masculino
gmmat_wrapper="gmmat_wrapper.Rscript"
SmkCes_male_models="/caminho_para_seus_dados/model_null_SmkCes_10pcs_male.rds"
genofile="/caminho_para_seus_dados/cohort_name.allchr-merge"
outfile="/caminho_para_seus_dados/SmkCes_male_score.txt"
# executando o script
Rscript ${gmmat_wrapper} --nullmodel_path=${SmkCes_male_models} \
--genofile=${genofile} \
--remove_fid=FALSE \
--outfile=${outfile}

## feminino
gmmat_wrapper="gmmat_wrapper.Rscript"
SmkCes_female_models="/caminho_para_seus_dados/model_null_SmkCes_10pcs_female.rds"
genofile="/caminho_para_seus_dados/cohort_name.allchr-merge"
outfile="/caminho_para_seus_dados/SmkCes_female_score.txt"
# executando o script
Rscript ${gmmat_wrapper} --nullmodel_path=${SmkCes_female_models} \
--genofile=${genofile} \
--remove_fid=FALSE \
--outfile=${outfile}
